<a href="https://colab.research.google.com/github/andy-nguyen-21/gold-market-data-vietnam/blob/main/sjc_gold_price_data_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# ==== Install required libraries ====
import requests
import csv
from datetime import date, timedelta, datetime
import os
from github import Github
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.colab import userdata

In [17]:
# ==== CONFIG ====
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
GITHUB_USER = userdata.get('GITHUB_USER')
REPO_NAME = "gold-market-data-vietnam"
FILE_PATH = "sjc_gold_prices.csv"

# Time range
start = date(2016, 1, 1)
end = date.today()
delta = timedelta(days=1)

def parse_price(value):
    """Convert '126.000' → 126000 (int)"""
    if value is None:
        return None
    return int(value.replace(".", "").replace(",", ""))

def fetch_day(d: date):
    """Fetch data for a single day and return list of rows (may be empty)."""
    url = "https://edge-api.pnj.io/ecom-frontend/v1/get-gold-price-history"
    params = {"date": d.strftime("%Y%m%d")}
    rows = []
    try:
        resp = requests.get(url, params=params, timeout=10)
        if resp.status_code == 200:
            data = resp.json()
            locations = data.get("locations", [])
            for loc in locations:
                if loc.get("name") == "TPHCM":   # Only TPHCM
                    for gtype in loc.get("gold_type", []):
                        if gtype.get("name") == "SJC":  # Only SJC
                            for record in gtype.get("data", []):
                                updated = record.get("updated_at")
                                gia_mua = parse_price(record.get("gia_mua"))
                                gia_ban = parse_price(record.get("gia_ban"))
                                if updated and gia_mua and gia_ban:
                                    dstr, t = updated.split(" ")
                                    rows.append((dstr, t, gia_mua, gia_ban))
    except Exception as e:
        print(f"Error fetching {d}: {e}")
    return d, rows

# ===== MAIN =====
local_file = FILE_PATH
with open(local_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["date", "time", "gia_mua", "gia_ban"])

    last_price = None

    # Run requests in parallel
    all_days = []
    current = start
    while current <= end:
        all_days.append(current)
        current += delta

    print(f"Fetching {len(all_days)} days...")

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(fetch_day, d): d for d in all_days}
        results = []
        for fut in as_completed(futures):
            d, rows = fut.result()
            results.append((d, rows))

    # Sort results by date
    results.sort(key=lambda x: x[0])

    # Process results day by day
    for d, rows in results:
        if rows:
            # Sort within day
            rows.sort(key=lambda x: datetime.strptime(x[1], "%H:%M:%S"))
            filtered = []
            last_in_day = None
            for row in rows:
                dstr, t, mua, ban = row
                current_price = (mua, ban)
                if current_price != last_in_day:
                    filtered.append((datetime.strptime(dstr, "%d/%m/%Y").strftime("%Y-%m-%d"), t, mua, ban))
                    last_in_day = current_price
            for row in filtered:
                writer.writerow(row)
            last_price = filtered[-1][2:]
        else:
            # No updates → carry forward yesterday's price
            if last_price:
                mua, ban = last_price
                writer.writerow([d.strftime("%Y-%m-%d"), "00:00:00", mua, ban])

print("✅ CSV created locally:", local_file)

# ==== Upload to GitHub ====
g = Github(GITHUB_TOKEN)
repo = g.get_repo(f"{GITHUB_USER}/{REPO_NAME}")

with open(local_file, "r", encoding="utf-8") as f:
    content = f.read()

try:
    contents = repo.get_contents(FILE_PATH)
    repo.update_file(contents.path, "Update gold price data", content, contents.sha, branch="main")
    print("✅ File updated on GitHub:", FILE_PATH)
except:
    repo.create_file(FILE_PATH, "Add gold price data", content, branch="main")
    print("✅ File created on GitHub:", FILE_PATH)


Fetching 3528 days...
✅ CSV created locally: sjc_gold_prices.csv
✅ File updated on GitHub: sjc_gold_prices.csv
